In [1]:
import pandas as pd
import psycopg2 as p2
from psycopg2 import sql
from collections import Counter

pd.set_option('display.width', 20000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 200)

## Попытка собрать валидационный датасет вручную

In [ ]:
dbname = "vulns_scanner"
user = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5432'


conn = p2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
cur = conn.cursor()
cur.execute('''
select * 
from cves c inner join descriptions d on c.cve_id_pk=d.cve_id_fk
inner join cve_cpe_config ccc on c.cve_id_pk=ccc.cve_id_fk inner join cpes cp on ccc.cpe_id_fk=cp.cpe_id_pk         
order by random()
limit 110;

''')
colnames = [desc[0] for desc in cur.description]
tuples = cur.fetchall()
cur.close()
# df = pd.DataFrame(tuples, columns=['cpe_id_pk', 'cpe_version', 'part', 'vendor', 
#                                     'product', 'version', 'update', 'edition', 
#                                     'sw_edition', 'target_sw', 'target_hw', 
#                                     'language', 'other', 'initial_cpe'])
df = pd.DataFrame(tuples, columns=colnames)
df.head()

,cve_id_pk,cve_id,descr_id,cve_id_fk,descr,cve_id_fk,cpe_id_fk,config_id,cpe_id_pk,cpe_version,part,vendor,product,version,update,edition,sw_edition,target_sw,target_hw,language,other,initial_cpe
0,17591,CVE-2022-0828,159361,17591,"The Download Manager WordPress plugin before 3.2.34 uses the uniqid php function to generate the master key for a download, allowing an attacker to brute force the key with reasonable resources gi...",17591,75723,30213,75723,2.3,a,wpdownloadmanager,wordpress_download_manager,3.1.02,,,,,wordpress,,,cpe:2.3:a:wpdownloadmanager:wordpress_download_manager:3.1.02:*:*:*:*:wordpress:*:*
1,5605,CVE-2010-4098,78305,5605,"monotone before 0.48.1, when configured to allow remote commands, allows remote attackers to cause a denial of service (crash) via an empty argument to the mtn command.",5605,32290,135003,32290,2.3,a,monotone,monotone,0.15,,,,,,,,cpe:2.3:a:monotone:monotone:0.15:*:*:*:*:*:*:*
2,132356,CVE-2021-29642,28399,132356,"GistPad before 0.2.7 allows a crafted workspace folder to change the URL for the Gist API, which leads to leakage of GitHub access tokens.",132356,621517,177944,621517,2.3,a,gistpad_project,gistpad,0.0.32,,,,,visual_studio,,,cpe:2.3:a:gistpad_project:gistpad:0.0.32:*:*:*:*:visual_studio:*:*
3,13622,CVE-2005-1274,111190,13622,Stack-based buffer overflow in the getIfHeader function in the WebDAV functionality in MySQL MaxDB before 7.5.00.26 allows remote attackers to execute arbitrary code via an HTTP unlock request and...,13622,52548,17812,52548,2.3,a,mysql,maxdb,-,,,,,,,,cpe:2.3:a:mysql:maxdb:-:*:*:*:*:*:*:*
4,21766,CVE-2022-24707,163536,21766,"Anuko Time Tracker is an open source, web-based time tracking application written in PHP. UNION SQL injection and time-based blind injection vulnerabilities existed in Time Tracker Puncher plugin ...",21766,125544,181554,125544,2.3,a,anuko,time_tracker,1.0.3,,,,,,,,cpe:2.3:a:anuko:time_tracker:1.0.3:*:*:*:*:*:*:*


In [42]:
df.descr.value_counts()

An issue was discovered in PartialReader in the uu_od crate before 0.0.4 for Rust. Attackers can read the contents of uninitialized memory locations via a user-provided Read operation.                                                                                                                                                                                                                                                                                                                            1
Auth. (admin+) Stored Cross-Site Scripting (XSS) vulnerability in NsThemes Advanced Social Pixel plugin <= 2.1.1 versions.                                                                                                                                                                                                                                                                                                                                                                                        

In [1]:
df

NameError: name 'df' is not defined

In [5]:
df = df.iloc[:100].drop_duplicates(subset='cve_id')

In [6]:
df.to_csv('df_to_annotate.csv', index=False)

In [54]:
len(df.descr.values)

100

In [30]:
with open('to_annotate_v2.txt', 'w') as file:
    for value in df['descr']:
        values = value.replace('\\n', ' ')
        file.write(f"{value}\n")

In [64]:
df.iloc[3].descr

'Cross-Site Request Forgery (CSRF) vulnerability in Jongmyoung Kim Korea SNS.This issue affects Korea SNS: from n/a through 1.6.3.\n\n'

In [2]:
df = pd.read_csv('df_to_annotate.csv')

In [4]:
df.iloc[:20]

,cve_id_pk,cve_id,descr_id,cve_id_fk,descr,cve_id_fk.1,cpe_id_fk,config_id,cpe_id_pk,cpe_version,part,vendor,product,version,update,edition,sw_edition,target_sw,target_hw,language,other,initial_cpe
0,17591,CVE-2022-0828,159361,17591,"The Download Manager WordPress plugin before 3.2.34 uses the uniqid php function to generate the master key for a download, allowing an attacker to brute force the key with reasonable resources gi...",17591,75723,30213,75723,2.3,a,wpdownloadmanager,wordpress_download_manager,3.1.02,NaN,NaN,NaN,NaN,wordpress,NaN,NaN,cpe:2.3:a:wpdownloadmanager:wordpress_download_manager:3.1.02:*:*:*:*:wordpress:*:*
1,5605,CVE-2010-4098,78305,5605,"monotone before 0.48.1, when configured to allow remote commands, allows remote attackers to cause a denial of service (crash) via an empty argument to the mtn command.",5605,32290,135003,32290,2.3,a,monotone,monotone,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:monotone:monotone:0.15:*:*:*:*:*:*:*
2,132356,CVE-2021-29642,28399,132356,"GistPad before 0.2.7 allows a crafted workspace folder to change the URL for the Gist API, which leads to leakage of GitHub access tokens.",132356,621517,177944,621517,2.3,a,gistpad_project,gistpad,0.0.32,NaN,NaN,NaN,NaN,visual_studio,NaN,NaN,cpe:2.3:a:gistpad_project:gistpad:0.0.32:*:*:*:*:visual_studio:*:*
3,13622,CVE-2005-1274,111190,13622,Stack-based buffer overflow in the getIfHeader function in the WebDAV functionality in MySQL MaxDB before 7.5.00.26 allows remote attackers to execute arbitrary code via an HTTP unlock request and...,13622,52548,17812,52548,2.3,a,mysql,maxdb,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:mysql:maxdb:-:*:*:*:*:*:*:*
4,21766,CVE-2022-24707,163536,21766,"Anuko Time Tracker is an open source, web-based time tracking application written in PHP. UNION SQL injection and time-based blind injection vulnerabilities existed in Time Tracker Puncher plugin ...",21766,125544,181554,125544,2.3,a,anuko,time_tracker,1.0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:anuko:time_tracker:1.0.3:*:*:*:*:*:*:*
5,147496,CVE-2017-18197,133338,147496,"In mxGraphViewImageReader.java in mxGraph before 3.7.6, the SAXParserFactory instance in convert() is missing flags to prevent XML External Entity (XXE) attacks, as demonstrated by /ServerView.",147496,668199,118247,668199,2.3,a,jgraph,mxgraph,3.4.1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:jgraph:mxgraph:3.4.1.2:*:*:*:*:*:*:*
6,99130,CVE-2018-19498,122429,99130,The Simplenia Pages plugin 2.6.0 for Atlassian Bitbucket Server has XSS.,99130,532139,109281,532139,2.3,a,simplenia,pages,2.4.4,NaN,NaN,NaN,NaN,bitbucket_server,NaN,NaN,cpe:2.3:a:simplenia:pages:2.4.4:*:*:*:*:bitbucket_server:*:*
7,4640,CVE-2010-2828,77340,4640,"Unspecified vulnerability in the H.323 implementation in Cisco IOS 12.1 through 12.4 and 15.0 through 15.1, and IOS XE 2.5.x before 2.5.2 and 2.6.x before 2.6.1, allows remote attackers to cause a...",4640,26824,83647,26824,2.3,o,cisco,ios,12.2scd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:o:cisco:ios:12.2scd:*:*:*:*:*:*:*
8,65236,CVE-2023-43193,16622,65236,Submitty before v22.06.00 is vulnerable to Cross Site Scripting (XSS). An attacker can create a malicious link in the forum that leads to XSS.,65236,374376,901342,374376,2.3,a,rcos,submitty,20.01.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:rcos:submitty:20.01.00:*:*:*:*:*:*:*
9,63737,CVE-2023-39976,15123,63737,log_blackbox.c in libqb before 2.0.8 allows a buffer overflow via long log messages because the header size is not considered.,63737,366659,858962,366659,2.3,a,clusterlabs,libqb,1.0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:clusterlabs:libqb:1.0.3:*:*:*:*:*:*:*


In [29]:
print(repr(df.iloc[11].descr))

'ScanCode.io is a server to script and automate software composition analysis with ScanPipe pipelines. Prior to version 32.5.1, the software has a possible command injection vulnerability in the docker fetch process as it allows to append malicious commands in the `docker_reference` parameter.\n\nIn the function `scanpipe/pipes/fetch.py:fetch_docker_image` the parameter `docker_reference` is user controllable. The `docker_reference` variable is then passed to the vulnerable function `get_docker_image_platform`.  However, the `get_docker_image_plaform` function constructs a shell command with the passed `docker_reference`. The `pipes.run_command` then executes the shell command without any prior sanitization, making the function vulnerable to command injections. A malicious user who is able to create or add inputs to a project can inject commands. Although the command injections are blind and the user will not receive direct feedback without logs, it is still possible to cause damage to

In [38]:
df.iloc[11:12].descr.str.replace('\n', ' ').values[0]

'ScanCode.io is a server to script and automate software composition analysis with ScanPipe pipelines. Prior to version 32.5.1, the software has a possible command injection vulnerability in the docker fetch process as it allows to append malicious commands in the `docker_reference` parameter.  In the function `scanpipe/pipes/fetch.py:fetch_docker_image` the parameter `docker_reference` is user controllable. The `docker_reference` variable is then passed to the vulnerable function `get_docker_image_platform`.  However, the `get_docker_image_plaform` function constructs a shell command with the passed `docker_reference`. The `pipes.run_command` then executes the shell command without any prior sanitization, making the function vulnerable to command injections. A malicious user who is able to create or add inputs to a project can inject commands. Although the command injections are blind and the user will not receive direct feedback without logs, it is still possible to cause damage to t

In [5]:
df[df.descr.str.contains('scanpipe')]

,cve_id_pk,cve_id,descr_id,cve_id_fk,descr,cve_id_fk.1,cpe_id_fk,config_id,cpe_id_pk,cpe_version,part,vendor,product,version,update,edition,sw_edition,target_sw,target_hw,language,other,initial_cpe
11,63541,CVE-2023-39523,14927,63541,"ScanCode.io is a server to script and automate software composition analysis with ScanPipe pipelines. Prior to version 32.5.1, the software has a possible command injection vulnerability in the do...",63541,365488,852176,365488,2.3,a,nexb,scancode.io,1.1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:nexb:scancode.io:1.1.0:*:*:*:*:*:*:*


In [39]:
df['descr_preprocessed'] = df['descr'].str.replace('\n', ' ')

In [44]:
df.iloc[10:20]

,cve_id_pk,cve_id,descr_id,cve_id_fk,descr,cve_id_fk.1,cpe_id_fk,config_id,cpe_id_pk,cpe_version,part,vendor,product,version,update,edition,sw_edition,target_sw,target_hw,language,other,initial_cpe,descr_preprocessed
10,105371,CVE-2013-1794,50416,105371,Buffer overflow in certain client utilities in OpenAFS before 1.6.2 allows remote authenticated users to cause a denial of service (crash) and possibly execute arbitrary code via a long fileserver...,105371,547837,78669,547837,2.3,a,openafs,openafs,1.5.68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:openafs:openafs:1.5.68:*:*:*:*:*:*:*,Buffer overflow in certain client utilities in OpenAFS before 1.6.2 allows remote authenticated users to cause a denial of service (crash) and possibly execute arbitrary code via a long fileserver...
11,63541,CVE-2023-39523,14927,63541,"ScanCode.io is a server to script and automate software composition analysis with ScanPipe pipelines. Prior to version 32.5.1, the software has a possible command injection vulnerability in the do...",63541,365488,852176,365488,2.3,a,nexb,scancode.io,1.1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:nexb:scancode.io:1.1.0:*:*:*:*:*:*:*,"ScanCode.io is a server to script and automate software composition analysis with ScanPipe pipelines. Prior to version 32.5.1, the software has a possible command injection vulnerability in the do..."
12,21495,CVE-2022-24329,163265,21495,"In JetBrains Kotlin before 1.6.0, it was not possible to lock dependencies for Multiplatform Gradle Projects.",21495,117256,163350,117256,2.3,a,jetbrains,kotlin,1.4.0,dev8042,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:jetbrains:kotlin:1.4.0:dev8042:*:*:*:*:*:*,"In JetBrains Kotlin before 1.6.0, it was not possible to lock dependencies for Multiplatform Gradle Projects."
13,69551,CVE-2023-5578,20937,69551,A vulnerability was found in Portábilis i-Educar up to 2.7.5. It has been declared as problematic. Affected by this vulnerability is an unknown functionality of the file \intranet\agenda_imprimir....,69551,408999,1007296,408999,2.3,a,portabilis,i-educar,2.0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:portabilis:i-educar:2.0.9:*:*:*:*:*:*:*,A vulnerability was found in Portábilis i-Educar up to 2.7.5. It has been declared as problematic. Affected by this vulnerability is an unknown functionality of the file \intranet\agenda_imprimir....
14,167,CVE-2004-0201,167,167,"Heap-based buffer overflow in the HtmlHelp program (hh.exe) in HTML Help for Microsoft Windows 98, Me, NT 4.0, 2000, XP, and Server 2003 allows remote attackers to execute arbitrary commands via a...",167,1493,3606,1493,2.3,o,microsoft,windows_nt,4.0,sp6a,enterprise_server,NaN,NaN,NaN,NaN,NaN,cpe:2.3:o:microsoft:windows_nt:4.0:sp6a:enterprise_server:*:*:*:*:*,"Heap-based buffer overflow in the HtmlHelp program (hh.exe) in HTML Help for Microsoft Windows 98, Me, NT 4.0, 2000, XP, and Server 2003 allows remote attackers to execute arbitrary commands via a..."
15,94343,CVE-2018-10938,117642,94343,A flaw was found in the Linux kernel present since v4.0-rc1 and through v4.13-rc4. A crafted network packet sent remotely by an attacker may force the kernel to enter an infinite loop in the cipso...,94343,515107,49024,515107,2.3,o,linux,linux_kernel,4.10,rc2,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:o:linux:linux_kernel:4.10:rc2:*:*:*:*:*:*,A flaw was found in the Linux kernel present since v4.0-rc1 and through v4.13-rc4. A crafted network packet sent remotely by an attacker may force the kernel to enter an infinite loop in the cipso...
16,142636,CVE-2017-10065,128478,142636,"Vulnerability in the Oracle Retail Point-of-Service component of Oracle Retail Applications (subcomponent: Security). Supported versions that are affected are 13.2, 13.3, 13.4, 14.0 and 14.1. Easi...",142636,656984,39038,656984,2.3,a,oracle,retail_point-of-service,6.0.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cpe:2.3:a:oracle:retail_point-of-service:6.0.10:*:*:*:*:*:*:*,"Vulnerability in the Oracle Retail Point-of-Service component of Oracle Retail Applications (subcompo

In [41]:
with open('to_annotate_no_newlines.txt', 'w') as file:
    for value in df['descr_preprocessed']:
        # values = value.replace('\\n', ' ')
        file.write(f"{value}\n")

In [15]:
import json
with open('/home/mikhail/Documents/pandan_study/vkr/vulns_scanner/mikhail_code/data/annotation_results/annotated_data.json') as f:
    r = json.loads(f.readline())

In [17]:
for i, m in enumerate(r):
    if m == []:
        print(m)
        print(i)
        break

In [18]:
len(r)

20

## Полный датасет для обучающей выборки из Stucco

В файле `make_dataset_nuner_v2.py` сделали из него нужный формат.

In [45]:
with open('/home/mikhail/Documents/pandan_study/vkr/vulns_scanner/mikhail_code/data/full_corpus.json') as j:
    corpus = json.loads(j.read())

In [47]:
len(corpus['NVD'])

15192

## Checking words for VERSION named entity

Какие слова бывают? Может, стоит сделать ручные эвристики для выделения версий? 

In [2]:
df_all = pd.read_csv('cve_dataset_bio_15192_texts.csv', index_col=False)

/tmp/ipykernel_23158/1254759156.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df_all = pd.read_csv('cve_dataset_bio_15192_texts.csv', index_col=False)


In [3]:
df_all

,cve_id,words,bio
0,CVE-2010-0001,Integer,B-relevant_term
1,0,underflow,I-relevant_term
2,0,in,O
3,0,the,O
4,0,unlzw,O
...,...,...,...
685105,0,2,O
685106,0,),O
685107,0,pg,O
685108,0,parameter,O


#### Распределение по годам в датасете Stucco

In [8]:
from collections import Counter
Counter(list(map(lambda x: x[:8], df_all.cve_id.unique())))

Counter({'CVE-2010': 4861,
         '0': 1,
         'CVE-2011': 4192,
         'CVE-2012': 4689,
         'CVE-2013': 1450})

In [23]:
all_versions = df_all[df_all.bio.isin(['B-version', 'I-version'])].words

In [24]:
len(set(all_versions))

5818

In [25]:
str_digits = [str(x) for x in range(10)]
str_digits

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [26]:
res = {}

for wrd, occ in Counter(all_versions).items():
    if not pd.isna(wrd):
        if len(set(wrd).intersection(set(str_digits))) == 0:
            res[wrd] = occ 

In [27]:
len(res)

173

In [31]:
dict(sorted(res.items(), key=lambda x: x[1], reverse=True)[:20])

{'before': 12899,
 'through': 1719,
 'and': 1652,
 'earlier': 1652,
 'Explorer': 280,
 'Anti-Virus': 39,
 'Express': 33,
 'Excel': 31,
 'flexible': 25,
 'Panels': 24,
 '.': 23,
 'XP': 18,
 'extension': 14,
 'TP': 12,
 'OP': 12,
 'MP': 12,
 'Comfort': 12,
 'Mobile': 12,
 'Extension': 9,
 'hotfix': 8}

Много мусора. Почистить?

Попробуем найти все "up to version"

In [39]:

df_all[(df_all.bio.isin(['B-version', 'I-version'])) & (df_all.words.str.contains('through'))]

,cve_id,words,bio
387,0,through,I-version
2005,0,through,I-version
2015,0,through,I-version
4286,0,through,I-version
4678,0,through,I-version
...,...,...,...
679932,0,through,I-version
680195,0,through,I-version
680633,0,through,I-version
680702,0,through,B-version


In [45]:
idx = 680195
df_all.iloc[idx-10:idx+10]

,cve_id,words,bio
680185,0,site,O
680186,0,",",O
680187,0,a,O
680188,0,different,O
680189,0,vulnerability,O
680190,0,than,O
680191,0,CVE-2013-2834,B-cve
680192,0,.,O
680193,CVE-2013-2944,strongSwan,B-application
680194,0,4.3.5,B-version


In [30]:
Counter(all_versions).most_common(40)

[('before', 12899),
 ('through', 1719),
 ('and', 1652),
 ('earlier', 1652),
 ('10.x', 480),
 ('6', 378),
 ('5.0', 352),
 ('4.x', 320),
 ('11.x', 315),
 ('Explorer', 280),
 ('7', 274),
 ('R2', 272),
 ('7.0', 270),
 ('3.x', 242),
 ('1.0', 236),
 ('9.x', 232),
 ('2.x', 227),
 ('6.0', 213),
 ('4.1', 200),
 ('8.x', 199),
 ('8.0', 195),
 ('2010', 183),
 ('2007', 176),
 ('2.0', 173),
 ('10.6', 170),
 ('4.0', 159),
 ('9.0', 147),
 ('8', 143),
 ('6.x', 141),
 ('9', 140),
 ('6.1', 139),
 ('5.1', 137),
 ('3.6.x', 131),
 ('7.x', 131),
 ('3.0', 128),
 ('1.1', 124),
 ('15.0', 123),
 ('9.1', 120),
 ('7.1', 119),
 ('2003', 118)]

In [67]:
list(filter(lambda x: any([str(i) for i in range(10)]) not in x[0], Counter(all_versions)))

TypeError: 'in <string>' requires string as left operand, not bool